## To-do
- Now that we have import and metadata extraction working, we need to start preprocessing (mostly interpolating timepoints for z-slices if recorded on frame-by-frame basis by the scope) and some scheme for identification of a nuclear and a spot channel that is compatible with switching between the two channels (e.g. using mCherry to segment nuclei during cycles but not at the division).
- Makes sense to use dask for visualization (e.g. choosing a threshold).
- Write DoG/segmentation fuction so that it can take either 2D or 3D data - give the option to segment off of a projection, or off of raw 3D data.
    - Write in options for DoG and LoG segmentation algorithm with standard nuclear sizes vs box DoG/LoG vs watershed.
        - Actually, box filtering might not be very helpful if we're cutting off part of the nucleus is z - the BP filtering will project it into a distorted gaussian if we're not right in the middle of the nucleus, and then misplace the centroid and botch the diameter estimation from $\sigma$. For 3D segmentation, it might be better to use a single filter to find markers then perform a watershed.
- 3D DoG notes:
    - $\sigma_{x, y} = 8$ works perfectly to segment out nuclei during nc 13.
    - $\sigma_z$ is BP-filtered (1, 9) where 9 is the Z-sigma corresponding to the whole nucleus. This allow the BP to be very permissive in Z and filter out the nuclei in x and y.
- Proposed procedure for local peak finding:
    - Run box DoG as below with permissive BP in z and LoG approximation in (x, y), only varying $\sigma$ in the latter.
    - Peak-finding on standard image (e.g. $\sigma_{x, y} = 8$), then use coordinates as initial guess for next sigma values.
- Simple BP filter + peak finding does a good job finding markers. Give option then to watershed segment directly off of the image, off of distance-transformed otsu thresholded image, and off of edge-finding.
    - For data with the mid-nuclear plane on the boundary of our z-stack, might be useful to give the option to segment in 2D, then threshold each nuclear column locally to identify the nucleus.
    - Need to write loop over timepoints, clean up small objects at each step, then commit segmentation to file.

In [1]:
from preprocessing.import_data import import_save_dataset

# from nuclear_segmentation import segment_nuclei
import napari

trim_series = True
lif_test_name = "test_data/2021-06-14/p2pdpwt"
lsm_test_name = "test_data/2023-04-07/p2pdp_zld-sites-ctrl_fwd_1"

(
    channels_full_dataset,
    original_global_metadata,
    original_frame_metadata,
    export_global_metadata,
    export_frame_metadata,
) = import_save_dataset(lsm_test_name, trim_series=trim_series, mode="tiff")

/home/ybadal/mambaforge/envs/transcription_pipeline/lib/python3.10/site-packages/pims/bioformats.py:410: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '
/home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/preprocessing/import_data.py:719: UserWarning: /home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/test_data/2023-04-07/p2pdp_zld-sites-ctrl_fwd_1/collated_dataset/collated_dataset_ch00.tiff is a low contrast image
  imsave(collated_data_path, channel_data, plugin="tifffile")
/home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/preprocessing/import_data.py:719: UserWarning: /home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/test_data/2023-04-07/p2pdp_zld-sites-ctrl_fwd_1/collated_dataset/collated_dataset_ch01.tiff is a low contrast image
  imsave(collated_data_path

In [2]:
nuclear_channel_metadata = export_frame_metadata[1]
nuclear_channel = channels_full_dataset[1]

In [3]:
viewer = napari.view_image(nuclear_channel, name="Nuclear Channel")
napari.run()

In [4]:
from nuclear_segmentation.segmentation import denoise_movie

In [5]:
from functools import partial
import numpy as np

In [6]:
func = partial(denoise_movie, denoising="gaussian", denoising_sigma=3)
func2 = lambda x: func(x[0])

In [7]:
%%time
test1 = func2([nuclear_channel])

CPU times: user 11.2 s, sys: 1.62 s, total: 12.8 s
Wall time: 12.7 s


In [8]:
from utils import parallel_computing

In [38]:
client.has_what()

{'tcp://127.0.0.1:32933': ('list-5541ef9004bc7e668e1be312c386a4d7',
  'ndarray-bc7081002ba08b5316730b33876121aa',
  'lambda-19b73dd26192ff486f01bbc388af69a4'),
 'tcp://127.0.0.1:33149': ('list-c6acb800f6d1979abe2091f5f59db5ce',
  'ndarray-88e203f0809ee6ff6b9aa0ffe02f4496',
  'lambda-d2cf91a6f57b80b422b87817cf1024e9'),
 'tcp://127.0.0.1:36427': ('list-a788e64623eec0e1b21a0d08ded53cd7',
  'ndarray-8190c8d234cf47448815029e54b7a06c',
  'lambda-2b3d8257ec7c324e79ed691430923880'),
 'tcp://127.0.0.1:36989': ('list-091fd71519d3b059df07c1dd5066bc4f',
  'ndarray-c7bb026ccce3a4d3fed0e78d3c46d0fc',
  'lambda-57fbf2865fe213bb7ba4894bad8648de'),
 'tcp://127.0.0.1:38315': ('list-55e974653330c92fec000d813fad900e',
  'ndarray-6a124e50255eba3119c2d4dfdb4273ff',
  'lambda-c0b016adb30211d0fa769d4278f310c5'),
 'tcp://127.0.0.1:39133': ('list-46bce1aa91f31ed238e2538a8db21c06',
  'ndarray-d8fe85a30da2fc1cb0d3a9f301e15214',
  'lambda-1f0bbe9d2659838cfccc7464aa051af1'),
 'tcp://127.0.0.1:43695': ('list-fe0090f83d9a9ae757bba56ce08c35e8',
  'ndarray-64886b31a016edd1966b0282a15ca5db',
  'lambda-af258084f8b80ddbe9a6c200c5dc3f38'),
 'tcp://127.0.0.1:46015': ('list-6eb8e93d7a2c0ea0710389e2defb72b1',
  'ndarray-738bad98dfc655df015440486640e364',
  'lambda-18c5a874e82d46b2b95551f2903f3d73')}

In [31]:
from importlib import reload

reload(parallel_computing)

<module 'utils.parallel_computing' from '/home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/utils/parallel_computing.py'>

In [9]:
movies_list = [nuclear_channel]
dtype_out = nuclear_channel.dtype

In [39]:
%%time
test2 = parallel_computing.send_compute_to_cluster(
    movies_list, func, {"client": client}, dtype_out
)

CPU times: user 2.13 s, sys: 6.92 s, total: 9.05 s
Wall time: 12.6 s


In [40]:
viewer.add_image(test2[0])

<Image layer 'Image [1]' at 0x7f28c1080520>

In [24]:
del test2

In [35]:
test2[0].shape

(168, 256, 512)

In [36]:
test2[1]

[[<Future: finished, type: numpy.ndarray, key: ndarray-d8fe85a30da2fc1cb0d3a9f301e15214>,
  <Future: finished, type: numpy.ndarray, key: ndarray-bc7081002ba08b5316730b33876121aa>,
  <Future: finished, type: numpy.ndarray, key: ndarray-88e203f0809ee6ff6b9aa0ffe02f4496>,
  <Future: finished, type: numpy.ndarray, key: ndarray-6a124e50255eba3119c2d4dfdb4273ff>,
  <Future: finished, type: numpy.ndarray, key: ndarray-c7bb026ccce3a4d3fed0e78d3c46d0fc>,
  <Future: finished, type: numpy.ndarray, key: ndarray-64886b31a016edd1966b0282a15ca5db>,
  <Future: finished, type: numpy.ndarray, key: ndarray-8190c8d234cf47448815029e54b7a06c>,
  <Future: finished, type: numpy.ndarray, key: ndarray-738bad98dfc655df015440486640e364>]]

In [25]:
client

<Client: 'tcp://127.0.0.1:8786' processes=8 threads=8, memory=29.80 GiB>

In [31]:
with Client(address="localhost:8786") as client:
    num_processes = len(client.scheduler_info()["workers"])

    # Figure out how to split movie into chunks to distribute across processes
    num_timepoints = movies_list[0].shape[0]
    num_timepoints_per_chunk = int(np.ceil(num_timepoints / num_processes))
    num_axes = len(movies_list[0].shape)
    chunk_shape = (num_timepoints_per_chunk,) + movies_list[0].shape[1:num_axes]

    # dask_movies_list = []
    # for movie in movies_list:
    #     dask_movies_list.append(_convert_to_dask(movie, chunk_shape))

    # map = da.map_blocks(func, *dask_movies_list, meta=np.array((), dtype=dtype_out))
    # processed_movie = map.compute()
    dask_movies_list = []
    for movie in movies_list:
        dask_movies_list.append(_convert_to_list(movie, chunk_shape))

    # Reshape list for scattering
    scatter_list = [["nan"] * len(dask_movies_list)] * len(dask_movies_list[0])
    for i, movie in enumerate(dask_movies_list):
        for j, chunk in enumerate(movie):
            scatter_list[j][i] = dask_movies_list[i][j]

    scattered_data = client.scatter(scatter_list)
    map = [client.submit(func2, distributed) for distributed in scattered_data]
    processed_movie = client.gather(map)
    processed_movie = np.concatenate(processed_movie, axis=0)

In [10]:
# from nuclear_segmentation import segmentation
# from tracking import track_features

import numpy as np
from dask.distributed import LocalCluster, Client

In [11]:
cluster = LocalCluster(
    host="localhost",
    scheduler_port=8786,
    threads_per_worker=1,
    n_workers=8,
    memory_limit="4GB",
)

In [12]:
cluster

LocalCluster(822ebcb2, 'tcp://127.0.0.1:8786', workers=8, threads=8, memory=29.80 GiB)

In [14]:
client = Client(cluster)

In [42]:
import dask

In [44]:
isinstance(13, dask.distributed.client.Client)

False

In [7]:
from importlib import reload

reload(segmentation)

<module 'nuclear_segmentation.segmentation' from '/home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/nuclear_segmentation/segmentation.py'>

In [6]:
denoised = segmentation.denoise_movie_parallel(
    nuclear_channel,
    denoising="gaussian",
    denoising_sigma=3,
    address="localhost:8786",
)

IndexError: list index out of range

In [7]:
denoised = segmentation.denoise_movie_parallel(
    nuclear_channel,
    denoising="gaussian",
    denoising_sigma=3,
    address="localhost:8786",
)

mask = segmentation.binarize_movie_parallel(
    denoised,
    thresholding="global_otsu",
    closing_footprint=segmentation.ellipsoid(3, 3),
    address="localhost:8786",
)

markers = segmentation.mark_movie_parallel(
    nuclear_channel,
    mask,
    low_sigma=[3, 5.5, 5.5],
    high_sigma=[10, 14.5, 14.5],
    max_footprint=((1, 25), segmentation.ellipsoid(3, 3)),
    max_diff=1,
    address="localhost:8786",
)

marker_coords = np.array(np.nonzero(markers)).T

labels = segmentation.segment_movie_parallel(
    denoised,
    markers,
    mask,
    watershed_method="raw",
    min_size=200,
    address="localhost:8786",
)

segmentation_dataframe = track_features.segmentation_df(
    labels,
    nuclear_channel,
    nuclear_channel_metadata,
)

tracked_dataframe = track_features.link_df(
    segmentation_dataframe,
    search_range=18,
    adaptive_stop=1,
    adaptive_step=0.99,
    memory=1,
    pos_columns=["x", "y"],
    t_column="frame_reverse",
    velocity_predict=True,
)

centroids = np.array(
    [
        [row["frame"] - 1, int(row["z"]), int(row["y"]), int(row["x"])]
        for _, row in tracked_dataframe.iterrows()
    ]
)
centroids = np.unique(centroids, axis=0)

# tracked_labels = track_features.reorder_labels_parallel(
#     labels,
#     tracked_dataframe,
#     address="localhost:8786",
# )

/home/ybadal/mambaforge/envs/transcription_pipeline/lib/python3.10/site-packages/distributed/client.py:3109: UserWarning: Sending large graph of size 876.75 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2023-07-06 18:56:53,561 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.64 GiB -- Worker memory limit: 3.73 GiB
2023-07-06 18:56:57,021 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 3.00 GiB -- Worker memory limit: 3.73 GiB
2023-07-06 18:57:01,464 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:42221 (pid=396698) exceeded 95% memory budget. Restarting...
2023-07-06 18:57:01,679 - distributed.nanny - WARNING - Restart

Using the rule of thumb $r \approx \sigma \sqrt{2} \ (2D)$ and $r \approx \sigma \sqrt{3} \ (3D)$ as rough bounds for the kernels used for band-pass filtering seems to net a perfect segmentation.

In [83]:
cluster.close()

In [36]:
viewer.add_points(marker_coords)

<Points layer 'marker_coords' at 0x7ff440a35420>

In [8]:
viewer.add_labels(labels)

<Labels layer 'labels' at 0x7f9d4822afb0>

In [75]:
test_df = segmentation_df(labels, nuclear_channel, export_frame_metadata[1])

In [76]:
test_df

label          z           y           x  frame          t_s  t_frame  \
0          1  10.423652  190.495588  310.694034      1     8.240113        0   
1          2  11.121260  137.438135    2.569475      1     8.791586        0   
2          3  11.545987   54.576450  413.024117      1     9.127342        0   
3          4  11.158082  125.477688  423.591774      1     8.820695        0   
4          5  10.842528  159.222024  502.247449      1     8.571243        0   
...      ...        ...         ...         ...    ...          ...      ...   
23586    166  16.325758    9.353474  477.487450    167  2849.429195      171   
23587    167  17.107577  118.305752    2.253921    167  2850.047239      171   
23588    168  17.600033    1.487592  142.162745    167  2850.436536      171   
23589    169  17.908525    1.431623  334.909033    167  2850.680405      171   
23590    172  18.022608  253.834370  239.830822    167  2850.770590      171   

       frame_reverse  t_frame_reverse  
0                167              171  
1                167              171  
2                167              171  
3                167              171  
4                167              171  
...              ...              ...  
23586              1                0  
23587              1                0  
23588              1                0  
23589              1                0  
23590              1                0  

[23591 rows x 9 columns]

In [70]:
linked_df = link_dataframe(
    test_df,
    search_range=18,
    adaptive_stop=1,
    adaptive_step=0.99,
    memory=1,
    pos_columns=["x", "y"],
    t_column="frame_reverse",
    velocity_predict=True,
)

Frame 167: 41 trajectories present.


In [71]:
linked_df

label          z           y           x  frame          t_s  t_frame  \
0          1  14.909759  210.269216  504.405988    167  2848.309818      171   
1          2  14.958818  158.286138  456.895890    167  2848.348601      171   
2          3  14.825406  174.249374  157.196094    167  2848.243136      171   
3          4  14.718019  215.220187  271.829508    167  2848.158244      171   
4          5  14.889322  239.022783   62.278645    167  2848.293663      171   
...      ...        ...         ...         ...    ...          ...      ...   
23586     39  13.309906  242.117774   29.797088      1    10.521756        0   
23587     41  13.624967   35.222641    4.736303      1    10.770818        0   
23588     42  14.084977  113.976705   68.034117      1    11.134466        0   
23589     43  13.532374  143.066172  246.001071      1    10.697622        0   
23590     44  14.374468   79.080045   26.438397      1    11.363315        0   

       frame_reverse  t_frame_reverse  particle  
0                  1                0         1  
1                  1                0         2  
2                  1                0         3  
3                  1                0         4  
4                  1                0         5  
...              ...              ...       ...  
23586            167              171       267  
23587            167              171       140  
23588            167              171       100  
23589            167              171       119  
23590            167              171       262  

[23591 rows x 10 columns]

In [72]:
reordered = reorder_labels(labels, linked_df)

In [73]:
viewer.add_labels(reordered)

<Labels layer 'reordered' at 0x7f9ddecda080>

- Trackpy seems to work pretty well. There is a frame (at the interface between two series) where there is a slight jump in positions it isn't able to deal with. This should be fixable with a use of the predictor method.
- There's also a pretty significant mistracking of 3-4 nuclei when they start getting very close together. Either dropping the search range in nc14 or doing tracking both forward and in reverse with velocity prediction should help.

In [69]:
viewer.add_points(centroids)

<Points layer 'centroids' at 0x7f9de7a71f60>

In [8]:
import pandas as pd

print(pd.__version__)

2.0.2


In [9]:
tracked_dataframe

label          z           y           x  frame          t_s  t_frame  \
0          1  14.909759  210.269216  504.405988    167  2848.309818      171   
1          2  14.958818  158.286138  456.895890    167  2848.348601      171   
2          3  14.825406  174.249374  157.196094    167  2848.243136      171   
3          4  14.718019  215.220187  271.829508    167  2848.158244      171   
4          5  14.889322  239.022783   62.278645    167  2848.293663      171   
...      ...        ...         ...         ...    ...          ...      ...   
23586     39  13.309906  242.117774   29.797088      1    10.521756        0   
23587     41  13.624967   35.222641    4.736303      1    10.770818        0   
23588     42  14.084977  113.976705   68.034117      1    11.134466        0   
23589     43  13.532374  143.066172  246.001071      1    10.697622        0   
23590     44  14.374468   79.080045   26.438397      1    11.363315        0   

       frame_reverse  t_frame_reverse  particle  
0                  1                0         1  
1                  1                0         2  
2                  1                0         3  
3                  1                0         4  
4                  1                0         5  
...              ...              ...       ...  
23586            167              171       267  
23587            167              171       140  
23588            167              171       100  
23589            167              171       119  
23590            167              171       262  

[23591 rows x 10 columns]

In [19]:
from functools import partial

test_reorder = track_features.reorder_labels

In [11]:
test = test_reorder(labels)

In [12]:
viewer.add_labels(test)

<Labels layer 'test' at 0x7f35d05abcd0>

In [15]:
from utils.parallel_computing import send_compute_to_cluster
import numpy as np

In [16]:
test_llel = send_compute_to_cluster(
    [labels], test_reorder, {"address": "localhost:8786"}, np.uint32
)

2023-07-06 15:13:41,920 - distributed.worker - WARNING - Compute Failed
Key:       ('reorder_labels-2bb99c59879d169d983ade8f6499e967', 6, 0, 0, 0)
Function:  subgraph_callable-f5c9cdb8-6785-4502-898c-51dae162
args:      (array([[[[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0

IndexError: index 166 is out of bounds for axis 0 with size 21

In [24]:
from utils.parallel_computing import _parallelization_kwargs

In [17]:
np.arange(0, 167, 14)

array([  0,  14,  28,  42,  56,  70,  84,  98, 112, 126, 140, 154])

In [32]:
test_llel = send_dataframe_compute_to_cluster(
    [labels], tracked_dataframe, test_reorder, {"address": "localhost:8786"}, np.uint32
)

ValueError: left side of old and new divisions are different

In [49]:
from importlib import reload

reload(track_features)

<module 'tracking.track_features' from '/home/ybadal/Documents/Berkeley/github_repositories/transcription_pipeline/tracking/track_features.py'>

In [50]:
tracked_labels = track_features.reorder_labels_parallel(
    labels,
    tracked_dataframe,
)

TypeError: Delayed objects of unspecified length are not iterable

In [48]:
viewer.add_labels(tracked_labels)

<Labels layer 'tracked_labels [1]' at 0x7f35e8501e10>

In [7]:
testb = [["nan"] * 2] * 8

In [10]:
print(*testb)

['nan', 'nan'] ['nan', 'nan'] ['nan', 'nan'] ['nan', 'nan'] ['nan', 'nan'] ['nan', 'nan'] ['nan', 'nan'] ['nan', 'nan']


In [9]:
testb[6][1]

'nan'

In [97]:
import numpy as np
import dask.array as da
import zarr
import multiprocessing as mp
import dask
from dask.distributed import LocalCluster, Client


def _convert_to_dask(movie, chunk_shape):
    """
    Converts supported array types to dask with requested chunking.
    """
    if isinstance(movie, np.ndarray):
        dask_movie = da.from_array(movie, chunks=chunk_shape)
    elif isinstance(movie, zarr.core.Array):
        dask_movie = da.from_zarr(movie, chunks=chunk_shape)
    elif isinstance(movie, da.Array):
        dask_movie = da.rechunk(movie, chunks=chunk_shape)
    else:
        raise Exception("Movie data type not recognized, must be numpy, zarr or dask.")

    return dask_movie


def _parallelization_kwargs(kwargs):
    """
    Constructs Dask Client kwarg dict for connecting to existing LocalCluster or
    creating new LocalCluster. Default cluster parameters are enforced here.
    """
    existing_client_kwargs = {}
    new_client_kwargs = {}

    try:
        existing_client_kwargs["client"] = kwargs["client"]
    except KeyError:
        existing_client_kwargs["client"] = None

    try:
        existing_client_kwargs["address"] = kwargs["address"]
        existing_client_kwargs["timeout"] = "2s"
    except KeyError:
        existing_client_kwargs["address"] = None

    try:
        num_processes = kwargs["num_processes"]
    except KeyError:
        num_processes = 4

    try:
        new_client_kwargs["memory_limit"] = kwargs["memory_limit"]
    except KeyError:
        new_client_kwargs["memory_limit"] = "4GB"

    new_client_kwargs["n_workers"] = int(min(0.9 * mp.cpu_count(), num_processes))
    new_client_kwargs["processes"] = True
    new_client_kwargs["threads_per_worker"] = 1

    return existing_client_kwargs, new_client_kwargs


def _compute(movies_list, func, dtype_out, client):
    """
    Dispatches the data in movies_list, with each movie chunked so that each worker
    gets one chunk of each of the movies in movies_list. This data is then operate
    on using func by the workers. Returns a tuple of the result of the operation
    (expected to be a single numpy array of the same shape as any of the movies
    in movies_list) and the scattered data as futures objects for later computations
    without needing to serialize again.
    """
    num_processes = len(client.scheduler_info()["workers"])

    # Creates a list of movies indexed by position in movies_list then chunk
    chunked_movies = []
    for movie in movies_list:
        chunked_movies.append(np.array_split(movie, num_processes))

    # Reshape list for scattering so that scatter_list is indexed by chunk
    # then by position in movies_list
    scatter_list = [["nan"] * len(chunked_movies)] * len(chunked_movies[0])
    for i, movie in enumerate(chunked_movies):
        for j, chunk in enumerate(movie):
            scatter_list[j][i] = chunk

    scattered_data = client.scatter(scatter_list)
    map = [client.submit(func, distributed) for distributed in scattered_data]
    processed_movie = client.gather(map)
    processed_movie = np.concatenate(processed_movie, axis=0)

    return processed_movie, scattered_data, scatter_list


def _compute_map(movies_list, func, client_kwargs, dtype_out):
    """
    Runs a function func taking a list of identically-shaped arrays as the only
    positional argument in parallel across workers on a Dask client generated as per
    options in the client_kwargs dict. The input arrays need to be packed in
    respective order as a list before passing to this function.
    """
    try:
        client = client_kwargs["client"]
        if not isinstance(client, dask.distributed.client.Client):
            raise TypeError("Provided client is not a Dask client object.")

        processed_movie, scattered_data, scatter_list = _compute(
            movies_list, func, dtype_out, client
        )

    # If no client provided, start new client in context manager
    except KeyError as _:
        with Client(**client_kwargs) as client:
            processed_movie, scattered_data = _compute(
                movies_list, func, dtype_out, client
            )
            scattered_data = None

    return processed_movie, scattered_data, scatter_list


def send_compute_to_cluster(movies_list, func, kwargs, dtype_out):
    """
    Sends func parallel computation to existing client and LocalCluster if available,
    falls back to creating one if not. Uses kwargs dictionary passed down from parent
    function to determine parallelization parameters.

    :param list movies_list: List of movies that need to be dispatched to the
        LocalCluster in the order in which they are provided to func as positional
        arguments.
    :param function func: Function to apply to each chunk of the input movies in
        movies_list, with a list of the i-th blocks of each movie in movie_list
        being provided to func as its only positional arguments in the order in
        which they are provided in movies_list, parallelized across the LocalCluster.
    :param dict kwargs: Dictionary of all keyword arguments used to determine the
        parameters of the LocalCluster and parallelization. Accepts key-value
        `client` to specify a Dask client to send the computation to, as well as
        keyword arguments for the following Dask LocalCluster parameters in case
        creation of a new LocalCluster is needed:
        *`address`
        *`timeout`
        *`num_processes`
        *`memory_limit`
        *`n_workers`
        *`processes`
        *`threads_per_worker`
    :param dtype_out: Expected datatype of output of func.
    :type dtype_out: Numpy datatype of form np.array((), dtype).
    :return: Output of func.
    :rtype: Output dtype of func, usually a numpy array.
    """
    existing_client_kwargs, new_client_kwargs = _parallelization_kwargs(kwargs)

    # Try to connect to provided address. If no address or cannot connect, spin
    # up local cluster
    if (
        existing_client_kwargs["address"] is None
        and existing_client_kwargs["client"] is None
    ):
        func_eval = _compute_map(movies_list, func, new_client_kwargs, dtype_out)
    else:
        try:
            func_eval = _compute_map(
                movies_list, func, existing_client_kwargs, dtype_out
            )
        except OSError:
            warnings.warn(
                "".join(
                    [
                        "Cluster not found at specified address, ",
                        "starting new cluster with default parameters.",
                    ]
                ),
                stacklevel=2,
            )
            func_eval = _compute_map(movies_list, func, new_client_kwargs, dtype_out)

    processed_movie, scattered_data, scatter_list = func_eval

    return processed_movie, scattered_data, scatter_list

In [98]:
%%time
test3 = send_compute_to_cluster(movies_list, func2, {"client": client}, dtype_out)

CPU times: user 1.27 s, sys: 3.42 s, total: 4.69 s
Wall time: 8.11 s


In [99]:
viewer.add_image(test3[0])

<Image layer 'Image' at 0x7efccc13c280>

In [77]:
scatter_list = test3[2]

In [80]:
movie = scatter_list[0]

In [88]:
rejoined = [submovie[0] for submovie in scatter_list]

In [126]:
viewer.add_image(rejoined)

<Image layer 'rejoined' at 0x7efcc7f06f80>

In [87]:
len(scatter_list)

8

In [91]:
rejoined = np.concatenate(rejoined)

In [93]:
test_split = np.array_split(nuclear_channel, 8)

In [125]:
viewer.add_image(np.concatenate(test_split))

<Image layer 'Image' at 0x7efd080f1150>

In [106]:
movies_list = [nuclear_channel, nuclear_channel]

In [107]:
chunked_movies = []
for movie in movies_list:
    chunked_movies.append(np.array_split(movie, num_processes))

In [109]:
len(chunked_movies[0])

8

In [121]:
# scatter_list = [["nan"] * len(chunked_movies)] * len(chunked_movies[0])
scatter_list = [["nan" for i in range(2)] for j in range(8)]
for i in range(8):
    for j in range(2):
        scatter_list[i][j] = chunked_movies[j][i]

In [122]:
rejoined = [scatter_item[0] for scatter_item in scatter_list]
rejoined = np.concatenate(rejoined)